In [1]:
import pandas as pd
import numpy as np

from time import sleep
from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [2]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
raw_df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1HOUR,
                                      str(365) + ' days ago UTC'))

# First 6 column represent Time and OHLCV values
raw_df = raw_df.iloc[:, :6]
raw_df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
raw_df['Time'] = pd.to_datetime(raw_df['Time'], unit='ms')
for column in raw_df.columns:
    if column != 'Time':
        raw_df[column] = raw_df[column].astype(float)

# Set Time as index
raw_df.set_index('Time', inplace=True)
raw_df = raw_df.iloc[:-1]

# Generate Technical Indicators

In [3]:
def get_ta(raw_df):
    df = raw_df.copy()
    df['SMA50'] = df['Close'].rolling(50).mean()
    df['SMA200'] = df['Close'].rolling(200).mean()
    df['RSI'] = momentum.rsi(df['Close'])
    df['MACD'] = trend.macd(df['Close'])
    df['MACD_sig'] = trend.macd_signal(df['Close'])
    df['MACD_diff'] = trend.macd_diff(df['Close'])
    df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
    df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
    df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
    df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])
    df.dropna(inplace=True)
    return df


In [4]:
df = get_ta(raw_df)

# Generate Trading Signals

For each TAs, a buy, hold and sell signal will be generated. They will bear the weight of 1, 0 and -1 respectively

In [5]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI
Time,,,,,,,,,,,,,,,
2023-08-03 23:00:00,29235.25,29238.93,29193.64,29193.64,399.79392,29320.6750,29287.49955,47.185407,4.826251,-0.496192,5.322443,0.001566,17.004727,21.649655,20.412189
2023-08-04 00:00:00,29193.65,29251.68,29166.66,29232.94,780.63159,29320.2540,29287.15210,50.009875,4.121322,0.427311,3.694011,0.002911,16.287705,22.196485,19.305295
2023-08-04 01:00:00,29232.95,29234.00,29191.04,29198.15,403.83160,29310.0972,29286.79835,47.583884,0.746789,0.491206,0.255583,0.001598,15.621899,21.560277,18.751956
2023-08-04 02:00:00,29198.15,29209.91,29100.00,29166.00,1033.66855,29296.9766,29285.74085,45.392451,-4.470258,-0.501087,-3.969172,0.000979,15.931412,26.044819,17.379467
2023-08-04 03:00:00,29166.00,29178.70,29136.17,29161.29,540.30101,29283.3048,29284.53765,45.065011,-8.882472,-2.177364,-6.705108,-0.000626,16.218817,25.273954,16.865076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-25 10:00:00,64306.71,64314.01,63973.46,64165.68,834.35878,65733.4250,65981.07820,34.046191,-528.046968,-478.012610,-50.034358,-0.025726,38.496473,30.922254,8.471224
2024-07-25 11:00:00,64165.68,64284.01,64089.72,64096.00,1115.62453,65676.5810,65973.04990,33.253750,-524.434265,-487.296941,-37.137324,-0.026104,39.817564,29.954239,8.206034
2024-07-25 12:00:00,64096.01,64400.00,63900.24,64124.00,1628.67274,65621.6608,65964.51990,33.919341,-513.393723,-492.516298,-20.877426,-0.026920,41.287975,30.589425,7.551212


In [6]:
def get_signal(df):

    #SMA Golden Cross & Death Cross
    sma_buy_cond = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) < df['SMA200'])
    sma_sell_cond = (df['SMA50'] < df['SMA200']) & (df['SMA50'].shift(1) > df['SMA200'])
    df['Signal_sma'] = np.where(sma_buy_cond, 1, np.where(sma_sell_cond, -1, 0))

    # MACD Below 0 Cross
    macd_buy_cond = (df['MACD_diff'] > 0) & (df['MACD_diff'].shift(1) < 0) & (df['MACD'] < 0) &(df['MACD_sig'] < 0)
    macd_sell_cond = (df['MACD_diff'] < 0) & (df['MACD_diff'].shift(1) > 0) & (df['MACD'] > 0) &(df['MACD_sig'] > 0)
    df['Signal_macd'] = np.where(macd_buy_cond, 1, np.where(macd_sell_cond, -1, 0))

    # RSI Overbought & Oversold
    rsi_buy_cond = (df['RSI'] > 30) & (df['RSI'].shift(1) < 30)
    rsi_sell_cond = (df['RSI'] < 70) & (df['RSI'].shift(1) > 70)
    df['Signal_rsi'] = np.where(rsi_buy_cond, 1, np.where(rsi_sell_cond, -1, 0))

    # ADX Trend Confirmation
    adx_buy_cond = (df['ADX'] > 25) & (df['+DI'] > df['-DI'])
    adx_sell_cond = (df['ADX'] > 25) & (df['-DI'] > df['+DI'])
    df['Signal_adx'] = np.where(adx_buy_cond, 1, np.where(adx_sell_cond, -1, 0))

    # Momentum Trade
    momentum_buy_cond = (df['Momentum'] > 0) & (df['Momentum'].shift(1) > 0) & (df['Momentum'].shift(2) > 0) & (df['Momentum'].shift(3) > 0) & (df['Momentum'].shift(4) > 0) \
                        & (df['Momentum'].shift(5) > 0) & (df['Momentum'].shift(6) > 0) & (df['Momentum'].shift(7) > 0) & (df['Momentum'].shift(8) > 0) & (df['Momentum'].shift(9) > 0)
    momentum_sell_cond = (df['Momentum'] < 0) & (df['Momentum'].shift(1) < 0) & (df['Momentum'].shift(2) < 0) & (df['Momentum'].shift(3) < 0) & (df['Momentum'].shift(4) < 0) \
                        & (df['Momentum'].shift(5) < 0) & (df['Momentum'].shift(6) < 0) & (df['Momentum'].shift(7) < 0) & (df['Momentum'].shift(8) < 0) & (df['Momentum'].shift(9) < 0)
    df['Signal_momentum'] = np.where(momentum_buy_cond, 1, np.where(momentum_sell_cond, -1, 0))

    df['Buy/Sell Consensus'] = df['Signal_sma'] + df['Signal_macd'] + df['Signal_rsi'] + df['Signal_adx'] + df['Signal_momentum']

    return df

In [7]:
df = get_signal(df)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,...,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi,Signal_adx,Signal_momentum,Buy/Sell Consensus
Time,,,,,,,,,,,,,,,,,,,,,
2023-08-03 23:00:00,29235.25,29238.93,29193.64,29193.64,399.79392,29320.6750,29287.49955,47.185407,4.826251,-0.496192,...,0.001566,17.004727,21.649655,20.412189,0,0,0,0,0,0
2023-08-04 00:00:00,29193.65,29251.68,29166.66,29232.94,780.63159,29320.2540,29287.15210,50.009875,4.121322,0.427311,...,0.002911,16.287705,22.196485,19.305295,0,0,0,0,0,0
2023-08-04 01:00:00,29232.95,29234.00,29191.04,29198.15,403.83160,29310.0972,29286.79835,47.583884,0.746789,0.491206,...,0.001598,15.621899,21.560277,18.751956,0,0,0,0,0,0
2023-08-04 02:00:00,29198.15,29209.91,29100.00,29166.00,1033.66855,29296.9766,29285.74085,45.392451,-4.470258,-0.501087,...,0.000979,15.931412,26.044819,17.379467,0,0,0,0,0,0
2023-08-04 03:00:00,29166.00,29178.70,29136.17,29161.29,540.30101,29283.3048,29284.53765,45.065011,-8.882472,-2.177364,...,-0.000626,16.218817,25.273954,16.865076,-1,0,0,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-25 10:00:00,64306.71,64314.01,63973.46,64165.68,834.35878,65733.4250,65981.07820,34.046191,-528.046968,-478.012610,...,-0.025726,38.496473,30.922254,8.471224,0,0,0,-1,-1,-2
2024-07-25 11:00:00,64165.68,64284.01,64089.72,64096.00,1115.62453,65676.5810,65973.04990,33.253750,-524.434265,-487.296941,...,-0.026104,39.817564,29.954239,8.206034,0,0,0,-1,-1,-2
2024-07-25 12:00:00,64096.01,64400.00,63900.24,64124.00,1628.67274,65621.6608,65964.51990,33.919341,-513.393723,-492.516298,...,-0.026920,41.287975,30.589425,7.551212,0,0,0,-1,-1,-2


# Run Test Trades on Historcical Data

In [8]:
historical_df = df[df['Buy/Sell Consensus'] != 0]
historical_df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,...,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi,Signal_adx,Signal_momentum,Buy/Sell Consensus
Time,,,,,,,,,,,,,,,,,,,,,
2023-08-04 03:00:00,29166.00,29178.70,29136.17,29161.29,540.30101,29283.3048,29284.53765,45.065011,-8.882472,-2.177364,...,-0.000626,16.218817,25.273954,16.865076,-1,0,0,0,0,-1
2023-08-04 04:00:00,29161.28,29209.21,29134.70,29205.98,420.45225,29271.2072,29282.67710,48.836260,-8.673096,-3.476510,...,0.000685,16.044686,23.937251,18.138762,-1,0,0,0,0,-1
2023-08-04 13:00:00,29222.54,29333.08,29208.27,29240.72,1520.05490,29204.9796,29272.61125,52.961853,-5.608666,-7.841537,...,0.006767,12.199374,17.965714,26.676984,0,1,0,0,0,1
2023-08-04 17:00:00,29265.39,29269.24,29197.20,29200.01,758.35684,29195.4238,29268.54015,47.698852,4.058522,-0.914889,...,0.001853,11.318817,21.486485,21.886666,0,0,0,0,1,1
2023-08-05 03:00:00,29086.04,29102.45,29082.25,29094.53,430.05398,29177.0162,29255.00915,44.974885,-35.581371,-32.776881,...,-0.005448,25.784206,32.488874,14.889020,0,0,0,-1,-1,-2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-25 10:00:00,64306.71,64314.01,63973.46,64165.68,834.35878,65733.4250,65981.07820,34.046191,-528.046968,-478.012610,...,-0.025726,38.496473,30.922254,8.471224,0,0,0,-1,-1,-2
2024-07-25 11:00:00,64165.68,64284.01,64089.72,64096.00,1115.62453,65676.5810,65973.04990,33.253750,-524.434265,-487.296941,...,-0.026104,39.817564,29.954239,8.206034,0,0,0,-1,-1,-2
2024-07-25 12:00:00,64096.01,64400.00,63900.24,64124.00,1628.67274,65621.6608,65964.51990,33.919341,-513.393723,-492.516298,...,-0.026920,41.287975,30.589425,7.551212,0,0,0,-1,-1,-2


In [9]:
balance = 1000
open_trade = False
bought_price = 0
# sell_price = 0
coin = 0

In [10]:
for idx, row in historical_df.iterrows():
    if (row['Buy/Sell Consensus'] >= 2) & (open_trade == False):
        coin = (balance / row['Close']) *0.999
        bought_price = row['Close']
        open_trade = True
    elif (row['Buy/Sell Consensus'] <= -2) & (open_trade == True):
        open_trade = False
        print(f"PnL:{(row['Close']-bought_price)/bought_price * 100}")
        balance = coin * row['Close'] *0.999

PnL:-2.064272812910123
PnL:-0.5811474274025049
PnL:-0.43594123937550555
PnL:-0.961584349247738
PnL:-1.8281851075985587
PnL:0.8161880670702112
PnL:3.6446758906559706
PnL:-0.7801320206728518
PnL:26.66736781001343
PnL:-0.37111703639237975
PnL:4.501794165506847
PnL:-2.8569409914083215
PnL:-2.4712501615309947
PnL:-0.06560376239052165
PnL:14.67976366243981
PnL:-4.62065713265652
PnL:-1.4579873160448351
PnL:0.4515956924288818
PnL:-2.029125346527538
PnL:-1.6350823410144273
PnL:-2.805338941471094
PnL:2.5357173872174137
PnL:-2.7780660545973204
PnL:-1.3106624322889444
PnL:14.746034840622505
PnL:22.706302838763058
PnL:4.368010289088295
PnL:-6.624048820736145
PnL:5.6128839433511235
PnL:2.2684726920291087
PnL:-5.161179938646545
PnL:-2.7042173902549704
PnL:6.69416482115294
PnL:-3.2121000391632295
PnL:-2.052649122250086
PnL:3.238354576124288
PnL:-2.090006784881639
PnL:0.8087015743747492
PnL:-2.983511117454115
PnL:-1.4237920886208315
PnL:-1.6141278972252853
PnL:-0.32163742690058483
PnL:-3.86056130148591

In [11]:
1 - 1/1000

0.999

# Run Test Trades on Live Data

In [12]:
raw_df

,Open,High,Low,Close,Volume
Time,,,,,
2023-07-26 16:00:00,29324.01,29369.55,29271.23,29302.43,1480.34290
2023-07-26 17:00:00,29302.43,29325.00,29243.24,29268.90,959.90410
2023-07-26 18:00:00,29268.91,29499.00,29230.01,29377.50,4709.75871
2023-07-26 19:00:00,29377.50,29401.94,29195.00,29401.93,2214.61506
2023-07-26 20:00:00,29401.93,29690.00,29390.00,29578.09,4595.91189
...,...,...,...,...,...
2024-07-25 10:00:00,64306.71,64314.01,63973.46,64165.68,834.35878
2024-07-25 11:00:00,64165.68,64284.01,64089.72,64096.00,1115.62453
2024-07-25 12:00:00,64096.01,64400.00,63900.24,64124.00,1628.67274


In [13]:
balance = 1000
open_trade = False
bought_price = 0
# sell_price = 0
coin = 0
balance_df = {f"{raw_df.index[-1]}": balance}

In [14]:
while True:
        # Get new hour data
        hour_data = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                                Client.KLINE_INTERVAL_1HOUR,
                                        str(1) + ' hour ago UTC'))

        # First 6 column represent Time and OHLCV values
        hour_data = hour_data.iloc[:, :6]
        hour_data.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

        # Convert first column into datetime
        hour_data['Time'] = pd.to_datetime(hour_data['Time'], unit='ms')
        for column in hour_data.columns:
                if column != 'Time':
                        hour_data[column] = hour_data[column].astype(float)

        # Set Time as index
        hour_data.set_index('Time', inplace=True)

        # Join with old data
        raw_df = pd.concat([raw_df, hour_data])

        # Get TAs and Trading Signals
        df = get_ta(raw_df)
        df = get_signal(df)

        # Simulate Trade
        if (df.iloc[-2, -1] >= 2) & (open_trade == False):
                coin = (balance / df.iloc[-1,0]) *0.999
                balance = 0
                bought_price = df.iloc[-1,0]
                open_trade = True
        elif (df.iloc[-2, -1] <= -2) & (open_trade == True):
                open_trade = False
                print(f"PnL:{(df.iloc[-1,0]-bought_price)/bought_price * 100}")
                balance = coin * df.iloc[-1,0] *0.999

        balance_df[f'{df.index[-1]}'] =  balance
        print(f"Current Balance: {balance} USD and {coin} BTC at {df.index[-1]}")

        sleep(3600)
     

Current Balance: 1000 USD and 0 BTC at 2024-07-25 15:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-25 16:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-25 17:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-25 18:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-25 19:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-25 20:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-25 21:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-25 22:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-25 23:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-26 00:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-26 01:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-26 02:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-26 03:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-26 04:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-26 05:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-26 06:00:00
Current Balance: 1000 USD and 0 BTC at 2024-07-26 07:00: